In [ ]:
import numpy as np
import panel as pn
import plotly.graph_objs as go

pn.extension('plotly')

In [ ]:
def plot_spacetime_surface(x_grid, y_grid, t_grid):
    surface = go.Surface(x=x_grid, y=y_grid, z=t_grid)
    layout = go.Layout(
        title='Lorentz transform of a 2-D square',
        autosize=False,
        width=600,
        height=500,
        margin=dict(t=50, b=50, r=50, l=50),
        scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1)),
        scene = dict(
            xaxis = dict(nticks=8, range=[-10,10],),
            yaxis = dict(nticks=8, range=[-10,10],),
            zaxis = dict(nticks=8, range=[-5e-8,5e-8],),
            aspectratio = dict(x=1, y=1, z=1)
        ),
    )
    fig = dict(data=[surface], layout=layout)
    plotly_pane = pn.pane.Plotly(fig)
    return plotly_pane

In [ ]:
xx = np.linspace(-1, 1, 2)
yy = np.linspace(-1, 1, 2)
x, y = np.meshgrid(xx, yy)
t = np.zeros((2, 2))

In [ ]:
from lorentz_transform import lorentz_transform


def plot_transformed_spacetime_surface(velocity_magnitude=0.0, velocity_angle=0.0):
    velocity = velocity_magnitude * np.array([
        np.cos(velocity_angle),
        np.sin(velocity_angle)
    ])
    
    positions = np.stack([x, y], axis=-1)
    pos_transformed, t_transformed = lorentz_transform(
        positions,
        t,
        velocity,
        speed_of_light=300_000_000

    )
    x_transformed = pos_transformed[:,:,0].squeeze()
    y_transformed = pos_transformed[:,:,1].squeeze()
    
    return plot_spacetime_surface(x_transformed, y_transformed, t_transformed)

In [ ]:
import panel.widgets as pnw

#velocity_magnitude  = pnw.FloatSlider(name='velocity_magnitude', value=0, start=0, end=0.99, step=0.05)
velocity_magnitude  = pnw.FloatSlider(name='velocity_magnitude', value=0, start=0, end=298_000_000, step=100_000)
velocity_angle  = pnw.FloatSlider(name='velocity_angle', value=0, start=0, end=2*np.pi)


@pn.depends(velocity_magnitude, velocity_angle)
def reactive_plot_transformed_spacetime_surface(velocity_magnitude, velocity_angle):
    return plot_transformed_spacetime_surface(velocity_magnitude, velocity_angle)

widgets   = pn.Column(velocity_magnitude, velocity_angle)
transformed_plot = pn.Row(reactive_plot_transformed_spacetime_surface, widgets)
transformed_plot